In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import MobileNetV2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
TRAIN_DATA_PATH = "drive/MyDrive/train"

# Create an image data generator object
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Define the batch size
batch_size = 32

# Load the training data
train_generator = datagen.flow_from_directory(
    TRAIN_DATA_PATH,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# Load the validation data
validation_generator = datagen.flow_from_directory(
    TRAIN_DATA_PATH,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 818 images belonging to 3 classes.
Found 202 images belonging to 3 classes.


In [11]:
# Load the pre-trained MobileNetV2 model, excluding the top layer
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze the base_model
base_model.trainable = False

# Create a new model on top
model = Sequential([
  base_model,
  Flatten(),
  Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 flatten_1 (Flatten)         (None, 62720)             0         
                                                                 
 dense_2 (Dense)             (None, 3)                 188163    
                                                                 
Total params: 2446147 (9.33 MB)
Trainable params: 188163 (735.01 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [12]:
# Define the number of epochs
epochs = 10

# Fit the data to the model
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples // batch_size,
    epochs = epochs)

Epoch 1/10
25/25 [==============================] - 10s 278ms/step - loss: 4.7612 - accuracy: 0.6730 - val_loss: 0.8013 - val_accuracy: 0.8542
Epoch 2/10
25/25 [==============================] - 7s 290ms/step - loss: 0.4330 - accuracy: 0.9504 - val_loss: 0.5062 - val_accuracy: 0.9167
Epoch 3/10
25/25 [==============================] - 7s 294ms/step - loss: 0.0602 - accuracy: 0.9822 - val_loss: 0.7449 - val_accuracy: 0.8750
Epoch 4/10
25/25 [==============================] - 8s 330ms/step - loss: 0.0228 - accuracy: 0.9962 - val_loss: 0.3284 - val_accuracy: 0.9167
Epoch 5/10
25/25 [==============================] - 6s 244ms/step - loss: 0.0073 - accuracy: 0.9975 - val_loss: 0.2834 - val_accuracy: 0.9271
Epoch 6/10
25/25 [==============================] - 7s 288ms/step - loss: 0.0082 - accuracy: 0.9975 - val_loss: 0.5045 - val_accuracy: 0.9062
Epoch 7/10
25/25 [==============================] - 6s 236ms/step - loss: 2.3472e-04 - accuracy: 1.0000 - val_loss: 0.2056 - val_accuracy: 0.9531
E

In [13]:
model.save('rps-model.h5')